<a href="https://colab.research.google.com/github/kimdongyeob/gitpractice/blob/master/cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#cuda exercises
#I don't have nvidia gpu....ㅜㅜㅜ
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-j4cg39b0
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-j4cg39b0
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=1501c615c89aad17ae277aeeb78858a0171835e192a3b51cc61ff2bda5f6a924
  Stored in directory: /tmp/pip-ephem-wheel-cache-_p2s_lwt/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
__global__ void hellocuda(void){
    printf("Hello, and fuck you\n");
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);

// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 8



In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void square(float *d_out, float *d_in) {
		int idx = threadIdx.x;
		float f = d_in[idx];
		d_out[idx]=f*f;

}

int main(int argc, char **argv){
		const int array_size=60;
		const int array_bytes = array_size*sizeof(float);

		float h_in[array_size];
		for (int i = 0; i<array_size; i++){
				h_in[i] = float(i);
		}
		float h_out[array_size];

		float *d_in;
		float *d_out;

		cudaMalloc((void**) &d_in, array_bytes);
		cudaMalloc((void**) &d_out, array_bytes);

		cudaMemcpy(d_in, h_in, array_bytes, cudaMemcpyHostToDevice);
	

		square<<<1,array_size>>>(d_out, d_in);
		cudaMemcpy(h_out, d_out, array_bytes, cudaMemcpyDeviceToHost);

		for (int i=0; i<array_size; i++){
				printf("%f", h_out[i]);
				printf(((i%4)!=3)? "\t":"\n");
		}

		printf("\n");

		for (int i=0; i<array_size; i++){
				printf("%f", h_in[i]);
				printf(((i%4)!=3)? "\t":"\n");
		}
		cudaFree(d_in);
		cudaFree(d_out);
		return 0;

}

0.000000	1.000000	4.000000	9.000000
16.000000	25.000000	36.000000	49.000000
64.000000	81.000000	100.000000	121.000000
144.000000	169.000000	196.000000	225.000000
256.000000	289.000000	324.000000	361.000000
400.000000	441.000000	484.000000	529.000000
576.000000	625.000000	676.000000	729.000000
784.000000	841.000000	900.000000	961.000000
1024.000000	1089.000000	1156.000000	1225.000000
1296.000000	1369.000000	1444.000000	1521.000000
1600.000000	1681.000000	1764.000000	1849.000000
1936.000000	2025.000000	2116.000000	2209.000000
2304.000000	2401.000000	2500.000000	2601.000000
2704.000000	2809.000000	2916.000000	3025.000000
3136.000000	3249.000000	3364.000000	3481.000000

0.000000	1.000000	2.000000	3.000000
4.000000	5.000000	6.000000	7.000000
8.000000	9.000000	10.000000	11.000000
12.000000	13.000000	14.000000	15.000000
16.000000	17.000000	18.000000	19.000000
20.000000	21.000000	22.000000	23.000000
24.000000	25.000000	26.000000	27.000000
28.000000	29.000000	30.000000	31.000000
32.000000	33.00

In [ ]:
%%cu
#include "cuda_runtime.h"
#include <stdio.h>
__global__ void helloCUDA(void){
    printf("Hello CuDA from GPU!\n");
}
int main(void){
    printf("Hello GPU\n");
    helloCUDA <<<1,10>>>();
    return 0;
}

Hello GPU



In [ ]:
%%cu
#include <stdio.h>
#define NUM_BLOCKS 16
#define BLOCK_WIDTH 1
__global__ void hello(){
    printf("hello world! I'm a thread %d, block %d\n", threadIdx.x, blockIdx.x);
}

int main(int argc, char **argv){
    hello<<<2,10>>>();
    cudaDeviceSynchronize();
    printf("That's all!\n");
    return 0;
}

hello world! I'm a thread 0, block 0
hello world! I'm a thread 1, block 0
hello world! I'm a thread 2, block 0
hello world! I'm a thread 3, block 0
hello world! I'm a thread 4, block 0
hello world! I'm a thread 5, block 0
hello world! I'm a thread 6, block 0
hello world! I'm a thread 7, block 0
hello world! I'm a thread 8, block 0
hello world! I'm a thread 9, block 0
hello world! I'm a thread 0, block 1
hello world! I'm a thread 1, block 1
hello world! I'm a thread 2, block 1
hello world! I'm a thread 3, block 1
hello world! I'm a thread 4, block 1
hello world! I'm a thread 5, block 1
hello world! I'm a thread 6, block 1
hello world! I'm a thread 7, block 1
hello world! I'm a thread 8, block 1
hello world! I'm a thread 9, block 1
That's all!

